<a href="https://colab.research.google.com/github/kelvinchiluwe/advanced_machine_learning_algorithms/blob/main/Copy_of_grayscale_encoder_decoder_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
# import gradio as gr
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint


In [ ]:
import gradio as gr

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [ ]:
(X_train, _), (X_test, _) = cifar10.load_data()


In [ ]:
image=(X_train, y_train), (X_test, y_test)

In [ ]:
X_gray = tf.image.rgb_to_grayscale(X_train)
X_test_gray = tf.image.rgb_to_grayscale(X_test)


In [ ]:
# 1. Encoder
inputs = Input(shape=(32, 32, 1))
x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
h = MaxPooling2D((2, 2))(x)
z = Conv2D(64, (3, 3), activation='relu', padding='same')(h)
encoded = MaxPooling2D((2, 2))(z)

# 2. Decoder
i = UpSampling2D((2, 2))(encoded)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(i)
j = UpSampling2D((2, 2))(x)
m = Conv2D(32, (3, 3), activation='relu', padding='same')(j)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(m)

autoencoder_model = tf.keras.Model(inputs, decoded)
autoencoder_model.compile(optimizer='adam', loss='mse')

autoencoder_model.summary()



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 3)      │           867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 75,075 (293.26 KB)

 Trainable params: 75,075 (293.26 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
autoencoder_model.fit(
    X_gray, X_train,
    validation_data=(X_test_gray, X_test),
    epochs=15,
    batch_size=128,
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=3, mode='min', restore_best_weights=True),
        ModelCheckpoint('gradiograyscale.keras',monitor='val_loss',save_best_only=True,mode='min',verbose=1)
    ]
)

Epoch 1/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 18407.4023
Epoch 1: val_loss improved from inf to 18631.08594, saving model to gradiograyscale.keras
391/391 ━━━━━━━━━━━━━━━━━━━━ 430s 1s/step - loss: 18407.4980 - val_loss: 18631.0859
Epoch 2/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 18452.0586
Epoch 2: val_loss did not improve from 18631.08594
391/391 ━━━━━━━━━━━━━━━━━━━━ 444s 1s/step - loss: 18452.0410 - val_loss: 18631.0859
Epoch 3/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 18411.6797
Epoch 3: val_loss did not improve from 18631.08594
391/391 ━━━━━━━━━━━━━━━━━━━━ 442s 1s/step - loss: 18411.7656 - val_loss: 18631.0859
Epoch 4/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 18464.2949
Epoch 4: val_loss did not improve from 18631.08594
391/391 ━━━━━━━━━━━━━━━━━━━━ 443s 1s/step - loss: 18464.2461 - val_loss: 18631.0859


In [ ]:


def prediction(image):
    img = image.resize((32, 32))
    img_array = tf.keras.utils.img_to_array(img)
    img_gray = tf.image.rgb_to_grayscale(img_array)

    img_batch = tf.expand_dims(img_gray, axis=0)


    output = autoencoder_model.predict(img_batch)[0]
    return np.clip(output, 0.0, 1.0)

interface = gr.Interface(
    fn=prediction,
    inputs=gr.Image(type='pil'),
    outputs=gr.Image(type='numpy'),
    title='Grayscale-to-Color Image Colorization',
    description='Upload a grayscale image and see how the model adds color!'
)

interface.launch(share=True)


In [ ]:
# Predict on test images
colorized_imgs = autoencoder_model.predict(X_test_gray)

# Show side-by-side comparison
import matplotlib.pyplot as plt

n = 5  # number of samples to display
plt.figure(figsize=(12, 6))
for i in range(n):
    # Grayscale input
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(X_test_gray[i].numpy().squeeze(), cmap='gray')
    ax.axis('off')
    if i == 0: ax.set_title('Grayscale Input')

    # Colorized output
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(colorized_imgs[i])
    ax.axis('off')
    if i == 0: ax.set_title('Colorized Output')

    # Ground truth color
    ax = plt.subplot(3, n, i + 1 + 2 * n)
    plt.imshow(X_test[i])
    ax.axis('off')
    if i == 0: ax.set_title('Original Color')

plt.tight_layout()
plt.show()
